In [1]:
#some basic imports and setups
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from difflib import SequenceMatcher



#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'memes')
#image_dir = current_dir

%matplotlib inline

In [2]:
from alexnet import AlexNet

#placeholder for input and dropout rate
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

#create model with default config ( == no skip_layer and 1000 units in the last layer)
model = AlexNet(x, keep_prob, 1000,[],['fc7','fc8'],512,weights_path='/data/alpv95/MemeProject/im2txt/bvlc_alexnet.npy') #maybe need to put fc8 in skip_layers

#define activation of last layer as score
score = model.fc6

#create op to calculate softmax 
#softmax = tf.nn.softmax(score)

Converting captions and meme vector representations into single Tfrecord

Requires putting memes through alexnet to find their vector rep, shuffling the captions, changing captions into their word2idx, finally saving one caption together with one meme.

In [3]:
#USE FOR ALEXNET !!!!!!!!!
#TRAINING SET

with open('/data/alpv95/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/data/alpv95/MemeProject/im2txt/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
deleters = []
for i,capt in enumerate(captions):
    if ' - ' not in capt or ' - -' in capt:
        deleters.append(i)
for i,delete in enumerate(deleters):
    del captions[delete-i]
data_memes = []
data_captions = []
data_meme_names = [] #just to check captions have been paired correctly
counter = 0
passed = 0


#Doing everything in one script: (the fc6 vectors are quite sparse)
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    
    for i,meme in enumerate(img_files):
        #meme_name = meme.replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')
        #meme_name = meme_name.replace('.jpg','').lower()
        #meme_name = meme_name.replace('-',' ')
        img = Image.open(meme)
        try:
            img.thumbnail((227, 227), Image.ANTIALIAS)
            #img = img.resize((227,227))
            #use img.thumbnail for square images, img.resize for non square
            assert np.shape(img) == (227, 227, 3)
        except AssertionError:
            img = img.resize((227,227))
            print('sizing error')
        
        # Subtract the ImageNet mean
        img = img - imagenet_mean #should probably change this
        
        # Reshape as needed to feed into model
        img = img.reshape((1,227,227,3))

        meme_vector = sess.run(score, feed_dict={x: img, keep_prob: 1}) #[1,4096]
        meme_vector = np.reshape(meme_vector,[4096])
        assert np.shape(meme_vector) == (4096,)
        
        match = []
        meme_name = captions[counter].split(' - ')[0]
        image_name = img_files[i].replace('/data/alpv95/MemeProject/im2txt/memes/','')
        image_name = image_name.replace('.jpg','')
        image_name = image_name.replace('-',' ')
        #print(meme_name,image_name)
        try:
            assert SequenceMatcher(a=meme_name.replace(' ',''),b=image_name.replace(' ','')).ratio() >= 0.75 or image_name in meme_name or meme_name in image_name
        except AssertionError:
            passed+=1
            continue
        
        while SequenceMatcher(a=meme_name.replace(' ',''),b=captions[counter].split(' - ')[0].replace(' ','')).ratio() >= 0.75 or meme_name in captions[counter].split(' - ')[0] or captions[counter].split(' - ')[0] in meme_name: 
            if counter==len(captions)-1:
                match.append(captions[counter].split(' - ')[-1])
                break
            elif captions[counter] == captions[counter].split(' - ')[-1]:
                counter += 1
            else:
                match.append(captions[counter].split(' - ')[-1])
                counter += 1
                
        
        #now save in tfrecords format, or prepare for that action
        meme_vectors = [meme_vector for cap in match]
        image_names = [image_name for cap in match]
        assert len(meme_vectors) == len(match)
        data_memes.extend(meme_vectors)
        data_captions.extend(match)
        data_meme_names.extend(image_names)

        if i % 100 == 0:
            print(i,len(data_memes),len(data_captions),len(data_meme_names))

print(passed)
print(len(data_memes))
        
 

('/data/alpv95/MemeProject/im2txt/memes/y-u-no.jpg', '/data/alpv95/MemeProject/im2txt/memes/mendozameme.jpg')
(0, 194, 194, 194)
(100, 19500, 19500, 19500)
sizing error
(200, 38642, 38642, 38642)
sizing error
sizing error
sizing error
(300, 57715, 57715, 57715)
sizing error
sizing error
(400, 76329, 76329, 76329)
sizing error
sizing error
(500, 94591, 94591, 94591)
sizing error
sizing error
(600, 113146, 113146, 113146)
sizing error
(700, 131507, 131507, 131507)
sizing error
(800, 149520, 149520, 149520)
sizing error
sizing error
sizing error
(900, 167518, 167518, 167518)
(1000, 185425, 185425, 185425)
sizing error
(1100, 203294, 203294, 203294)
(1200, 220336, 220336, 220336)
sizing error
(1300, 237823, 237823, 237823)
sizing error
sizing error
(1400, 254129, 254129, 254129)
sizing error
sizing error
sizing error
(1500, 270058, 270058, 270058)
(1600, 286622, 286622, 286622)
sizing error
sizing error
(1700, 302200, 302200, 302200)
sizing error
(1800, 316652, 316652, 316652)
sizing error

In [26]:
#USE FOR INCEPTION NETWORK (OR ALEXNET WITH FINETUNING) #NOW WITH LABELS INCLUDED
#TRAINING SET
#For this case just need to save image filenames alongside captions
with open('/data/alpv95/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/data/alpv95/MemeProject/im2txt/CaptionsClean.txt','r') as f:
    captions = f.readlines()

data_memes = []
data_captions = []
data_labels = []
counter = 0
passed = 0


for i,meme in enumerate(img_files):
    
    match = []
    meme_name = captions[counter].split(' - ')[0]
    image_name = meme.replace('/data/alpv95/MemeProject/im2txt/memes/','')
    image_name = image_name.replace('.jpg','')
    image_name = image_name.replace('-',' ')
    #print(meme_name,image_name)
    try:
        assert SequenceMatcher(a=meme_name.replace(' ',''),b=image_name.replace(' ','')).ratio() >= 0.75 or image_name in meme_name or meme_name in image_name
    except AssertionError:
        passed+=1
        continue
        
    while SequenceMatcher(a=meme_name.replace(' ',''),b=captions[counter].split(' - ')[0].replace(' ','')).ratio() >= 0.75 or meme_name in captions[counter].split(' - ')[0] or captions[counter].split(' - ')[0] in meme_name: 
        if counter==len(captions)-1:
            match.append(captions[counter].split(' - ')[-1])
            break
        elif captions[counter] == captions[counter].split(' - ')[-1]:
            counter += 1
        else:
            match.append(captions[counter].split(' - ')[-1])
            counter += 1
                
        
    #now save in tfrecords format, or prepare for that action
    meme_images = [meme for cap in match]
    meme_labels = [meme_name for cap in match]
    assert len(meme_images) == len(match)
    data_memes.extend(meme_images)
    data_captions.extend(match)
    data_labels.extend(meme_labels)

    if i % 100 == 0:
        print(i,len(data_memes),len(data_captions),len(meme_labels))
print(passed)
print(len(data_memes))
        

('/data/alpv95/MemeProject/im2txt/memes/y-u-no.jpg', '/data/alpv95/MemeProject/im2txt/memes/mendozameme.jpg')
('y u no', 'y u no')
(0, 194, 194, 194)
('willy wonka', 'willy wonka')
('the most interesting man in the world', 'the most interesting man in the world')
('futurama fry', 'futurama fry')
('success kid', 'success kid')
('one does not simply', 'one does not simply')
('bad luck brian', 'bad luck brian')
('first world problems', 'first world problems')
('philosoraptor', 'philosoraptor')
('grumpy cat ', 'grumpy cat')
('winter is coming', 'winter is coming')
('forever alone', 'forever alone')
('good guy greg', 'good guy greg')
('scumbag steve', 'scumbag steve')
('what if i told you', 'what if i told you')
('conspiracy keanu', 'conspiracy keanu')
('kermit the frog drinking tea', 'kermit the frog drinking tea')
('yo dawg', 'yo dawg')
('all the things', 'all the things')
('insanity wolf', 'insanity wolf')
('joseph ducreux', 'joseph ducreux')
('pedobear', 'pedobear')
('trollface', 'troll

('i will find you meme', 'i will find you meme')
('i double dare you', 'i double dare you')
('metal boy from hell', 'metal boy from hell')
('god', 'god')
('obama beer', 'obama beer')
('confused bill cosby ', 'confused bill cosby')
('minecraft guy', 'minecraft guy')
("i don't always guy meme", 'i dont always guy meme')
('buddy the elf', 'buddy the elf')
('dr. evil and his minions', 'dr evil and his minions')
('high/drunk guy', 'highdrunk guy')
('so i got that going on for me, which is nice', 'so i got that going on for me which is nice')
('liberal douche garofalo', 'liberal douche garofalo')
('everywhere', 'everywhere')
('angry cat meme', 'angry cat meme')
('x all the y', 'x all the y')
("you're gonna have a bad time", 'youre gonna have a bad time')
('mens wearhouse', 'mens wearhouse')
('socially awesome penguin', 'socially awesome penguin')
('jealous potteroman', 'jealous potteroman')
('kill yourself guy blank', 'kill yourself guy blank')
('nick cage', 'nick cage')
('tobey_maguire', 't

('old russian advisor', 'old russian advisor')
('fuck yeah band buffalo', 'fuck yeah band buffalo')
('ollie williams', 'o seaque pedo')
('ollie williams', 'ollie williams')
('south park wow guy', 'south park wow guy')
('obsessed girlfriend', 'obsessed girlfriend')
('the count from sesame street', 'the count from sesame street')
('no memory gandalf', 'no memory gandalf')
('english motherfucker, do you speak it?', 'english motherfucker do you speak it')
('spidermanwhisper', 'spidermanwhisper')
('birthday grumpy cat', 'birthday grumpy cat')
('yu no', 'yu no')
('do you even lift guy', 'do you even lift guy')
('sheltering suburban mom', 'sheltering suburban mom')
('slender man', 'slender man')
('reagan white house laughing', 'reagan white house laughing')
('ric flair', 'ric flair')
('cm punk apologize!', 'cm punk apologize')
('overly manly man', 'overly manly man')
('if i had one doubled', 'if i had one doubled')
('please guy', 'please guy')
('casos de familia', 'casos de familia')
('bane c

('abo', 'abo')
('oh you dog', 'oh you dog')
('honey booboo', 'honey booboo')
('rich raven', 'rich raven')
('dark daria', 'dark daria')
('jonnen neuvo', 'jonnen neuvo')
('ay ya', 'ay ya')
('patrick star instrument', 'patrick star instrument')
('meme tierno', 'meme tierno')
('ermahgerd berks', 'ermahgerd berks')
('mr angry cat', 'mr angry cat')
('praying otter', 'praying otter')
('right in the childhood man', 'right in the childhood man')
('retard horse', 'retard horse')
('comic book guy simpsons', 'comic book guy simpsons')
('breaking bad jesse', 'breaking bad jesse')
('the olympic queen', 'the olympic queen')
('rebecca black', 'rebecca black')
('uselibaba', 'uselibaba')
('born in it bane', 'born in it bane')
('profe el tambien estaba hablando', 'profe el tambien estaba hablando')
('lady gaga troll', 'lady gaga troll')
('celestia', 'celestia')
('disgusted ginger', 'disgusted ginger')
('aaand its gone', 'aaand its gone')
('true life', 'true life')
('drew pickles: the gayest man in the wo

('steven seagal', 'steven seagal')
('your x is bad and you should feel bad', 'your x is bad and you should feel bad')
('introspective pug', 'introspective pug')
('steve brule', 'steve brule')
('lawyer', 'lawyer')
('introvert cat', 'introvert cat')
('fernando torres bitchface', 'fernando torres bitchface')
('balotelli', 'balotelli')
('league of legends', 'league of legends')
('spongebob how tough am i?', 'spongebob how tough am i')
('jimmywtf', 'jimmywtf')
('ali g', 'ali g')
('vieja anciana pinochetista', 'vieja anciana pinochetista')
('survivor rat', 'survivor rat')
('scary black man', 'scary black man')
('no one ever pays me in gum', 'no one ever pays me in gum')
('nigga, you just went full retard', 'nigga you just went full retard')
('j walter weatherman', 'j walter weatherman')
('the non-molesting priest', 'the non molesting priest')
('neckbeard', 'neckbeard')
('good day ice cube', 'good day ice cube')
('business dog', 'business dog')
('anonymous memes', 'anonymous memes')
('che gue

...

('luka car', 'luka car')
('little peach', 'little peach')
('csi miami', 'csi miami')
('typical-madist', 'typical madist')
('typical human 2', 'typical human 2')
('fandom fox', 'fandom fox')
('typical snowboarder', 'typical snowboarder')
('chevy  chase', 'chevy chase')
('lumpy space princess', 'lumpy space princess')
('anime/anime', 'animeanime')
('haftbefehl', 'haftbefehl')
('psy gangnam style meme', 'psy gangnam style meme')
('tipical absurd', 'tipical absurd')
('tpc shit', 'tpc shit')
('truepainter', 'truepainter')
('epic food lady', 'epic food lady')
("it's colibritany!", 'its colibritany')
('bodybuilder problems', 'bodybuilder problems')
('vanga', 'vanga')
(1500, 270057, 270057, 106)
('koala roleador', 'koala roleador')
('annoying sports fan', 'annoying sports fan')
('zhirinovskiy88', 'zhirinovskiy88')
('typical_shatura', 'typical shatura')
('high school nerd', 'high school nerd')
('scumbag battlefield 3 guy', 'scumbag battlefield 3 guy')
('virgin gamer', 'virgin gamer')
('seemingl

('smugleaf', 'smugleaf')
('stoner dude', 'stoner dude')
('memoires_maniaque', 'memoires maniaque')
('bidolphin', 'bidolphin')
('facebook/yaomingfumon', 'facebookyaomingfumon')
('smartaleck', 'smartaleck')
('scumbag rio', 'scumbag rio')
('frustrated journalist cat', 'frustrated journalist cat')
('coreagues geologist', 'coreagues geologist')
('scumbag bmw', 'scumbag bmw')
('indiference maradona', 'indiference maradona')
('typical s.t.a.l.k.e.r.', 'typical stalker')
('pupil56', 'pupil56')
('shit just got real', 'shit just got real')
('gothiccrow', 'gothiccrow')
('close enough', 'close enough')
('malcolm tucker', 'malcolm tucker')
('neymarin', 'neymarin')
('annoying fb girl', 'annoying fb girl')
('snobbish', 'snobbish')
('typical rssu', 'typical rssu')
('indian metal guy', 'indian metal guy')
('overly attached boy friend', 'overly attached boy friend')
('zyzzlol', 'zyzzlol')
('fucked up flynn', 'fucked up flynn')
('juicy j', 'juicy j')
('mybike', 'mybike')
('guy gay', 'guy gay')
('smorodin

('kuban', 'kuban')
('typical fans "the x-files"', 'typical fans the x files')
('pinkie pie assualt', 'pinkie pie assualt')
('tookewl', 'tookewl')
('abby farle', 'abby farle')
('minecraft_dyshanbe', 'minecraft dyshanbe')
('ellino', 'ellino')
('typical fs fan', 'typical fs fan')
('missingno', 'missingno')
('pizza pizza john', 'stavropoltypical')
('pizza pizza john', 'pizza pizza john')
('draco hugs voldemort', 'draco hugs voldemort')
('belarus okay-man (ppc)', 'belarus okay man ppc')
('sad girl', 'sad girl')
('scumbag strache', 'scumbag strache')
('baked guy', 'baked guy')
('botota', 'botota')
('youmightbealibertarian', 'youmightbealibertarian')
('proffessor snape', 'proffessor snape')
('stoner dogs concerned friend', 'stoner dogs concerned friend')
('typical french department', 'typical french department')
('milan-rage', 'milan rage')
('vivo al limite', 'vivo al limite')
('drunk mentor', 'drunk mentor')
('trsreu', 'trsreu')
('darren criss doesnt know', 'darren criss doesnt know')
('chad

('notorious tfp', 'notorious tfp')
('el padrazo', 'el padrazo')
('amigofer', 'amigofer')
('omg run man', 'omg run man')
('facebook max', 'facebook max')
('hypocritical glee fandom', 'hypocritical glee fandom')
('batman killer dude', 'batman killer dude')
('harvest moon', 'harvest moon')
('barry the jew', 'barry the jew')
('celomm', 'celomm')
('depressive pinkamena', 'depressive pinkamena')
('average dota player', 'average dota player')
('kampfer become girl', 'kampfer become girl')
('sochi hyli', 'sochi hyli')
('karpin_mem', 'karpin mem')
('advice matt berninger', 'advice matt berninger')
('courtney love rant', 'courtney love rant')
('niccals', 'niccals')
('so hardcore guy', 'so hardcore guy')
('mr. carson', 'mr carson')
('epic level llama', 'epic level llama')
('lulz dot net', 'lulz dot net')
('umvc3 zero', 'umvc3 zero')
('typical vgasu', 'typical vgasu')
('mature debater', 'mature debater')
('rivera schatz', 'rivera schatz')
('rock fangirl kitty', 'rock fangirl kitty')
('sasha hater2

('careers', 'careers')
('ajiii', 'ajiii')
('mitch hedberg', 'mitch hedberg')
('granny-potterhad', 'granny potterhad')
('typical winx-arter', 'typical winx arter')
('tipicalnatsu', 'tipicalnatsu')
('qa pigeon', 'qa pigeon')
('the slenderman meme', 'the slenderman meme')
('meme mala suerte', 'meme mala suerte')
('brian scalabrine (white mamba)', 'brian scalabrine white mamba')
('typical_cuber', 'typical cuber')
('aromantic aardvark', 'aromantic aardvark')
('anders breivik scuba', 'anders breivik scuba')
('mormons meme', 'mormons meme')
('black samantha', 'black samantha')
('female internet troll ', 'female internet troll')
(2400, 401659, 401659, 189)
('active_elektrichker', 'active elektrichker')
('premed goat', 'premed goat')
('very optimistic smile', 'very optimistic smile')
('gary bettman', 'gary bettman')
('creepy tech support guy', 'creepy tech support guy')
('sergio ramos penalti', 'sergio ramos penalti')
('@nikoarrieta', 'vrn')
('@nikoarrieta', 'nikoarrieta')
('tipycal-anti-soviet

In [6]:
for i in range(10,50):
    print(data_memes[i*100+402000])
    print(data_captions[i*100+402000])
    print(data_labels[i*100+402000])

/data/alpv95/MemeProject/im2txt/memes/i-hardly-know-her.jpg
farsi? i hardly knew ye!

i hardly know her
/data/alpv95/MemeProject/im2txt/memes/typical-grandmother.jpg
te estas poniendo fuerte cabron

typical grandmother
/data/alpv95/MemeProject/im2txt/memes/shabbywag.jpg
i have a gf cos im white

shabbywag
/data/alpv95/MemeProject/im2txt/memes/facebook-roleplay-ocelot.jpg
how are you a demon and an angel?

facebook roleplay ocelot
/data/alpv95/MemeProject/im2txt/memes/facebook-roleplay-ocelot.jpg
happy happy new year  lindsay, richard & family! much <3 to you!

facebook roleplay ocelot
/data/alpv95/MemeProject/im2txt/memes/funny-stupid.jpg
seit der geburt alexist

funny stupid
/data/alpv95/MemeProject/im2txt/memes/unlucky-person.jpg
dulu saya punya penyakit muka jelek  setelah saya pergi ke klink tong fang malah mukanya dokter nya yang tambah jelek dan saya tambah ganteng terima kasih klinik tong fang

unlucky person
/data/alpv95/MemeProject/im2txt/memes/indian-gangster-wannabe.jpg
chec

In [27]:
data_labels[len(data_memes)-200]

'arashian alpaca'

In [23]:
#CREATE EVALUATION SET    #NOW WITH LABELS AS WELL
#SHOULD BE MEMES WITH REPEATED FORMAT 
for i in range(100):
    #print(data_memes[i*190].replace('/data/alpv95/MemeProject/im2txt/memes/',''),data_captions[i*190],i*190)
    print(data_meme_names[i*190],data_captions[i*190],i*190)

('y u no', 'meme generator users y u no give me more upvotes?\n', 0)
('y u no', 'y you no studying\n', 190)
('willy wonka', 'so you are in love? who is it this week?\n', 380)
('the most interesting man in the world', 'i dont always have sex but when i do its with your girlfriend\n', 570)
('futurama fry', 'not sure if beef or horse meat\n', 760)
('success kid', 'windows checking for a solution actually found a solution\n', 950)
('one does not simply', 'one does not simply watch all 3 movies in a row\n', 1140)
('bad luck brian', 'gets beat up in school suspended for "involvement"\n', 1330)
('first world problems', 'bit into a jelly donut got a dry piece\n', 1520)
('philosoraptor', "if a there's a zombie outbreak in vegas   does it stay in vegas?\n", 1710)
('grumpy cat', "sure i'll play with you... how about russian roulette\n", 1900)
('winter is coming', 'brace yourself walter is coming... and he is going to kick some ass\n', 2090)
('forever alone', 'happy birthday from geico insurance\n

In [28]:
eval_examples = [0,570,760,1140,2850,3040,4940,28500]
eval_captions = ['y u no','i dont always','not sure if','one does not simply','what if i told you','what if','so youre telling me','so then i said']
eval_labels = []
eval_memes = []

for idx in eval_examples:
    eval_memes.append(data_memes[idx])
    eval_labels.append(data_labels[idx])


In [29]:
eval_labels

['y u no',
 'the most interesting man in the world',
 'futurama fry',
 'one does not simply',
 'what if i told you',
 'conspiracy keanu',
 'skeptical 3rd world kid',
 'so then i said...']

In [30]:
#Should also look at removing examples which are similar but not exactly the same ##WITH LABELS

c = list(zip(data_memes, data_captions, data_labels))
no_repeats = []
# order preserving
def idfun(x): return x

seen = {}
no_repeats = []
for item in c:
    marker = idfun(item[1])
    # in old Python versions:
    # if seen.has_key(marker)
    # but in new ones:
    if marker in seen: continue
    seen[marker] = 1
    no_repeats.append(item)
print(len(data_captions))
len(no_repeats)


414388


385707

In [31]:
from random import shuffle
shuffle(no_repeats)
memes_shuffled, captions_shuffled, labels_shuffled = zip(*no_repeats)
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)
labels_shuffled = list(labels_shuffled)

In [50]:
import re
word_captions = []
for capt in captions_shuffled + labels_shuffled: #include labels_shuffled here for glove averages
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    word_captions.append(words)
print(len(word_captions))

771414


In [51]:
from collections import Counter
print("Creating vocabulary.")
counter = Counter()
for c in word_captions:
    counter.update(c)
print("Total words:", len(counter))

# Filter uncommon words and sort by descending count.
word_counts = [x for x in counter.items() if x[1] >= 3]
word_counts.sort(key=lambda x: x[1], reverse=True)
print("Words in vocabulary:", len(word_counts))

Creating vocabulary.
('Total words:', 150760)
('Words in vocabulary:', 41819)


In [52]:
# Create the vocabulary dictionary.
reverse_vocab = [x[0] for x in word_counts]
#unk_id = len(reverse_vocab)
vocab_dict = dict([(x, y) for (y, x) in enumerate(reverse_vocab)])

In [53]:
vocab_dict[':']

67

In [54]:
EMBEDDING_DIMENSION=300 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = current_dir

PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers
weights = []
index_counter = 0

with open('glove.42B.300d.txt','r') as file:
    for index, line in enumerate(file):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        if word in vocab_dict:
            index_counter += 1
            word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
            word2idx[word] = index_counter # PAD is our zeroth index so shift by one
            weights.append(word_weights)
        if index % 100000 == 0:
            print(index)
        if index + 1 == 2000000:
            break

EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random #maybe include start and end token here
UNKNOWN_TOKEN=len(weights)
#include * token as its very common? why not
word2idx['*'] = UNKNOWN_TOKEN
word2idx['UNK'] = UNKNOWN_TOKEN + 1
word2idx['<S>'] = UNKNOWN_TOKEN + 2
word2idx['</S>'] = UNKNOWN_TOKEN + 3
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]
print(VOCAB_SIZE)

#Save Vocabulary IF NEW
with tf.gfile.FastGFile('vocab_averages.txt', "w") as f:
    f.write("\n".join(["%s %d" % (w, c) for w, c in word2idx.iteritems()]))
print("Wrote vocabulary file:", 'vocab_averages.txt')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
38821
('Wrote vocabulary file:', 'vocab_averages.txt')


In [ ]:
weights[37984]

In [55]:
word2idx['_']

766

In [56]:
#SAVE EMBEDDING MATRIX IF NEW
np.savetxt('embedding_matrix_averages',weights)

In [57]:
import re
token_captions = []
for capt in captions_shuffled:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_caption.append(word2idx['</S>'])
    token_captions.append(token_caption)
    
#for training labels
token_labels = []
for capt in labels_shuffled:
    token_caption = []
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_labels.append(token_caption)
    
#for eval set
eval_tokens = []
for capt in eval_captions:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    #token_caption.append(word2idx['</S>'])
    eval_tokens.append(token_caption)
    
#for eval labels
token_labels_eval = []
for capt in eval_labels:
    token_caption = []
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_labels_eval.append(token_caption)

In [58]:
print(token_captions[330000])
print(captions_shuffled[330000])
print(token_labels[330000])
print(labels_shuffled[330000])
print(eval_tokens[3])
print(eval_captions[3])
print(token_labels_eval[3])
print(eval_labels[3])
word2idx['UNK']

[38819, 2965, 6463, 598, 65, 343, 5389, 407, 38820]
instant flame war just add penis size

[2965, 6463, 598]
instant flame war
[38819, 47, 138, 32, 717]
one does not simply
[47, 138, 32, 717]
one does not simply


38818

In [59]:
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)

In [60]:
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)

In [61]:
for i,ting in enumerate(deleters):
    print(ting)
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
    del labels_shuffled[ting-i]
    del token_labels[ting-i]
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)
len(deleters)

23533
38413
126877
132307
177383
199839
216832
235469
251932
317088
326004


0

In [62]:
many_unk = []
for i,capt in enumerate(token_captions):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        many_unk.append(i)
print(len(many_unk))

for i,ting in enumerate(many_unk):
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
    del labels_shuffled[ting-i]
    del token_labels[ting-i]

30209


In [63]:
unk_label = []
for i,capt in enumerate(token_labels):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        unk_label.append(i)
print(len(unk_label))

733


In [64]:
orig_unk_labels = []
for i in range(len(unk_label)):
    print(labels_shuffled[unk_label[i]])
    orig_unk_labels.append(labels_shuffled[unk_label[i]])
print(len(set(orig_unk_labels)))

ranzigen peelaert 
vandinha depressao
pan's labyrinth1
ranzigen peelaert 
pan's labyrinth1
vandinha depressao
assburgerin superneuvo
pan's labyrinth1
tipichnuy bntu
ranzigen peelaert 
tipichnuy bgtu
tipichnuy bgtu
pan's labyrinth1
ranzigen peelaert 
pan's labyrinth1
ranzigen peelaert 
pan's labyrinth1
tipichnuy bgtu
ranzigen peelaert 
pan's labyrinth1
tipichnuy bgtu
tipichnuy bntu
vandinha depressao
pan's labyrinth1
ranzigen peelaert 
ranzigen peelaert 
vandinha depressao
pan's labyrinth1
pan's labyrinth1
vandinha depressao
vandinha depressao
vandinha depressao
vaverka-perakladczyca
vandinha depressao
tipichnuy bntu
pan's labyrinth1
tipichnuy bgtu
typacal viewy
pan's labyrinth1
vandinha depressao
pan's labyrinth1
vaverka-perakladczyca
vandinha depressao
vaverka-perakladczyca
pan's labyrinth1
vandinha depressao
vaverka-perakladczyca
tipichnuy bntu
typacal viewy
pan's labyrinth1
tipichnuy bntu
pan's labyrinth1
tipichnuy bntu
vandinha depressao
tipichnuy bntu
vandinha depressao
vandinha d

In [65]:

def _int64_feature(value):
    """Wrapper for inserting an int64 Feature into a SequenceExample proto."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    """Wrapper for inserting a bytes Feature into a SequenceExample proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature_list(values):
    """Wrapper for inserting an int64 FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])


def _bytes_feature_list(values):
    """Wrapper for inserting a bytes FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_bytes_feature(v) for v in values])

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [41]:
#ONLY FOR ALEXNET 
memes_shuffled_int = []
for i,meme in enumerate(memes_shuffled):
    memes_shuffled_int.append(np.int_(meme*1000000000000000))
print(memes_shuffled_int[20][:100])

[ 9168698310852050                 0                 0                 0
                 0                 0                 0                 0
                 0                 0  2523229837417602                 0
                 0                 0                 0                 0
                 0                 0                 0   308641821146011
                 0                 0                 0  9107991218566894
                 0                 0  6078919887542725                 0
                 0                 0                 0                 0
  3697678327560425                 0                 0                 0
                 0                 0                 0                 0
                 0                 0                 0                 0
                 0  3752865076065063                 0                 0
                 0                 0  4150147914886474  3943297624588012
                 0                 0               

In [43]:
#ONLY FOR ALEXNET 
eval_memes_int = []
for i,meme in enumerate(eval_memes):
    eval_memes_int.append(np.int_(meme*1000000000000000))

In [66]:
#ONLY FOR INCEPTION
class ImageDecoder(object):
    """Helper class for decoding images in TensorFlow."""

    def __init__(self):
        # Create a single TensorFlow Session for all image decoding calls.
        self._sess = tf.Session()

        # TensorFlow ops for JPEG decoding.
        self._encoded_jpeg = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._encoded_jpeg, channels=3)

    def decode_jpeg(self, encoded_jpeg):
        image = self._sess.run(self._decode_jpeg,
                               feed_dict={self._encoded_jpeg: encoded_jpeg})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

In [42]:
#SAVING TRAINING SET (ALEXNET)
import sys
train_filename = 'train.tfrecordsALEX'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled_int)):
    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled_int))
        sys.stdout.flush()
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(memes_shuffled_int[i].tostring()), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/355437
Train data: 20000/355437
Train data: 40000/355437
Train data: 60000/355437
Train data: 80000/355437
Train data: 100000/355437
Train data: 120000/355437
Train data: 140000/355437
Train data: 160000/355437
Train data: 180000/355437
Train data: 200000/355437
Train data: 220000/355437
Train data: 240000/355437
Train data: 260000/355437
Train data: 280000/355437
Train data: 300000/355437
Train data: 320000/355437
Train data: 340000/355437


In [68]:
#SAVING TRAINING SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'train.tfrecordsAVERAGES'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled)):
    
    with tf.gfile.FastGFile(memes_shuffled[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i]),
          "train/labels": _int64_feature_list(token_labels[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/355487
Train data: 20000/355487
Train data: 40000/355487
Train data: 60000/355487
Train data: 80000/355487
Train data: 100000/355487
Train data: 120000/355487
Train data: 140000/355487
Train data: 160000/355487
Train data: 180000/355487
Train data: 200000/355487
Train data: 220000/355487
Train data: 240000/355487
Train data: 260000/355487
Train data: 280000/355487
Train data: 300000/355487
Train data: 320000/355487
Train data: 340000/355487


In [44]:
#SAVING EVAL SET (ALEXNET)
import sys
train_filename = 'eval.tfrecordsALEX'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(eval_memes_int)):
    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(eval_memes_int))
        sys.stdout.flush()
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(eval_memes_int[i].tostring()), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(eval_tokens[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/8


In [22]:
#SAVING EVAL SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'eval.tfrecords7'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(eval_memes)):
    
    with tf.gfile.FastGFile(eval_memes[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(eval_memes))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(eval_tokens[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/8


In [28]:
print(memes_shuffled[150003])
print(captions_shuffled[150003])

/data/alpv95/MemeProject/im2txt/memes/fuckmeright.jpg
i don't like cheese on my burgers fuck me, right?

